# Barplots and rank-abundance plots of all high-throughput cell-based pannings using alpaca library

Produces grids of barplots and whittaker plots for all selections in `panning-massive`. See analogous notebook for figures in [`panning-extended`: `fig-suppl-extended`](../../../panning-extended/workflow/notebooks/fig-suppl-extended.ipynb)

In [1]:
import os

# change working directory to `./panning-massive` for simplicity of access to feature tables, etc
# make sure we don't do this twice, or we'll end up in the wrong place and be very confused
if 'dir_changed' not in globals():
    os.chdir('../../')
    dir_changed = True

In [2]:
import pandas as pd, numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import plotnine
from plotnine import *

import svg_stack

In [3]:
import nbseq
import nbseq.viz.sample as nvs
import nbseq.viz.tree
import nbseq.ft as nbft
from nbseq.utils import replace_multiple
from nbseq.viz.utils import shorten_descriptions, extract_encoded_data_plotnine

In [4]:
# setup matplotlib to render figures as SVG with editable text preserved at appropriate font size
FONT_SIZE = 5

plt.rcParams.update({
    "font.family":"sans",
    "font.size": FONT_SIZE,
    "svg.fonttype": "none"
})

In [5]:
%config InlineBackend.figure_formats = ['svg']

***

In [6]:
ex = nbseq.Experiment.from_files(ft_aa=None,
                                 tree_cdr3=None, tree_aa=None) #'intermediate/cdr3/features/all/alpaca/asvs.nwk')
ex.obs['OprOP'] = np.nan

Loading experiment panning-massive from '/vast/palmer/home.mccleary/cng2/code/phageseq-paper/panning-massive'...
- Reading metadata from config/metadata_full.csv ...
- Reading phenotypes from config/phenotypes.csv ...
- Reading Config from config/config.yaml ...
- Warning: sqlite database '/vast/palmer/home.mccleary/cng2/code/phageseq-paper/panning-massive/intermediate/aa/asvs.db' does not exist
- Reading feature data for table 'cdr3' from results/tables/cdr3/asvs.csv (3.8 MB)...
- Reading cdr3 feature table from results/tables/cdr3/feature_table.biom (29.0 MB)...
- Warning: mmseqs2 database for space 'aa' at 'intermediate/aa/features_db/features' does not exist!
- Warning: mmseqs2 database for space 'cdr3' at 'intermediate/cdr3/features_db/features' does not exist!
- Reading enrichment model (conditional ECDF) for space cdr3 from results/tables/cdr3/enrichment/null/ecdf.pickle (307.6 kB)...
Finished in 0.78 seconds


In [7]:
ex

Experiment('panning-massive') with feature spaces ['cdr3']:
  obs: ['plate' 'well' 'depth' 'expt' 'round' 'sample' 'phage_library' 'notes'
    'r' 'io' 'kind' 'selection' 'replicate' 'name_full' 'name' 'well_027e'
    'sel_plate' 'sel_well' 'category' 'antigen' 'genotype_pair' 'gene_CS'
    'gene_S' 'genotype_CS' 'background_CS' 'strain_CS' 'loc_CS' 'cond_CS'
    'genotype_S' 'background_S' 'strain_S' 'loc_S' 'cond_S' 'cond_notes'  'bflm'
    'swim' 'twitch' 'swarm' 'PMB-R' 'FEP-R' 'TET-R' 'CIP-R' 'CHL-R'  'GEN-R'
    'ERY-R' 'IPM-R' 'cdiGMP' 'FliC' 'FliCa' 'FliCb' 'FlgEHKL' 'PilQ'  'PilA'
    'PilB' 'LasA' 'LasB' 'Apr' 'XcpQ' 'ToxA' 'EstA' 'LepA' 'PlpD'  'Phz' 'Pcn'
    'Pvd' 'Hcn' 'Rhl' 'T3SS' 'T6SS' 'Pel' 'Psl' 'CdrB' 'SCV'  'Mucoid'
    'Alginate' 'OprM' 'OprJ' 'OprN' 'OprP' 'OpdH' 'OprD' 'OprL'  'OprF' 'OprG'
    'OprH' 'OprB' 'MexAB' 'MexCD' 'MexEF' 'MexJK' 'MexXY'  'MexGHI' 'PirA' 'Pfu'
    'TonB' 'FptA' 'FpvA' 'PfeA' 'CupB5' 'CupA' 'CupB'  'CupC' 'CupD' 'LPS-
    LipidA-Palmito

In [8]:
import re
replacements = {
    re.compile(r"(\w+)\#[^\s:]+"): r"\1",
    "ZTP riboswitch":"...",
    "PAO397":"PAO1",
}
    
shorten_descriptions(ex.obs, replacements=replacements);
shorten_descriptions(ex.fts.cdr3.obs, replacements=replacements);
ex.fts.cdr3.obs['desc_short_ml'] = ex.fts.cdr3.obs['desc_short'].str.replace("/","\n")

for space in ['cdr3','aa']:
    if space in ex.fts:
        
        for field in ['genotype_CS','genotype_S']: 
            ex.fts[space].obs[field] = ex.fts[space].obs[field].fillna('')
            shorten_descriptions(ex.fts[space].obs, replacements, new_column=field, old_column=field)

        obs = ex.fts[space].obs
        obs.loc[obs['name'] == '027i.1.C8.1','background_S'] = 'PA103'
        obs.loc[obs['name'] == '027i.1.H3.1','background_S'] = 'PA14'
        obs.loc[obs['name'] == '027i.1.H3.1','description'] = obs.loc[obs['name'] == '027i.1.H3.1','description'].str.replace('?', 'PA14 WT')
        obs.loc[obs['name'] == '027i.1.C8.1','description'] = obs.loc[obs['name'] == '027i.1.C8.1','description'].str.replace('?','PA103 ∆fimX yfiR#PA1121::Tn.2(Tc)')

    _clinical_isolates = (obs['expt'] == '027i') & obs['background_S'].isna()
    obs.loc[_clinical_isolates,'antigen'] = 'clinical isolate'
    obs.loc[_clinical_isolates,'background_S'] = 'clinical isolate ' + obs.loc[_clinical_isolates,'strain_S']

In [9]:
def shorten(text):
    from nbseq.utils import replace_multiple
    # from nbseq.viz.utils import 
    return replace_multiple(pd.Series([text]), replacements)[0]

In [10]:
def arrayplot(files, path, ncol=6, nrow=8):
    import svg_stack as ss
    k = 0
    n = 0
    while k < len(files):
        n += 1       
        page = ss.VBoxLayout()
        for j in range(nrow):
            row = ss.HBoxLayout()
            for i in range(ncol):
                row.addSVG(files[k],alignment=ss.AlignTop|ss.AlignLeft)
                k += 1
                if k >= len(files):
                    break
                    
            page.addLayout(row)
            if k >= len(files):
                break
        
        doc = ss.Document()
        doc.setLayout(page)
        fn = path.format(page=n)
        doc.save(fn)
        print(fn)

## Barplots of high-throughput cell-based pannings using alpaca library (`fig-suppl-arrayed-barplots`)

In [11]:
def top_asv_barplot(ex, query, space='cdr3', n=30, select_from_round=4, phylo=False, **kwargs):
    samples = ex.query_ids(f"({query}) & kind =='+' & io == 'i'", space=space)
    if select_from_round is not None:
        top_from_samples = ex.query_ids(f"({query}) & r == {select_from_round} & kind =='+' & io == 'i'", space=space)
    else:
        top_from_samples = None
    
    
    ft_top = nbseq.viz.sample.collapse_top_asvs(ex.fts[space], samples, top_from_samples = top_from_samples, n=n)
    df = nbseq.ft.fortify(ft_top, obs=True, relative=True)
    
    if phylo:
        return nbseq.viz.sample.top_asv_plot_phylo(df, ex.tree[space], **kwargs)
    else: 
        return nbseq.viz.sample.top_asv_barplot(df, **kwargs)

In [12]:
def _plot_expt(ex, expt, library, plot, predicates='', 
               plot_path="results/plots/barplots/arrayed", 
               data_path="results/tables/figures/barplots/arrayed",
               title_pattern="{background_S} {antigen} - {method}",
               file_pattern="{background_S}_{antigen}_{method}.svg",
               show=True, sort=True,
               last_round=5,
               space='cdr3', **kwargs):
    query = f"expt == '{expt}' & phage_library == '{library}' & replicate == 1 & io == 'i' & kind == '+'"
    if predicates != '':
        query += f" & ({predicates})"
    ft = ex.query(query, space=space)
    expt_samples = ft.obs['sample'].unique()
    if sort:
        from natsort import natsorted
        expt_samples = natsorted(expt_samples)
        
    print(expt_samples)
    
    plot_paths = []
    data_paths = []
    for sample in expt_samples:
        
        # print(nbft.query(ft, f"sample == '{sample}'", axis='sample').obs.r)
        # print(nbft.query_ids(ft, f"sample == '{sample}' & r == '5'"))
        
        top_from_samples = nbft.query_ids(ft, f"sample == '{sample}' & r == {last_round}")

        # print("top_from_samples:")
        # print(top_from_samples)
        
        ft_expt = nvs.collapse_top_asvs(
            ft, 
            samples = nbft.query_ids(ft, f"sample == '{sample}'"),
            top_from_samples = top_from_samples,
            n = 40,
            relative=True)
        
        first_row = ft_expt.obs.iloc[0,:]
        # print("{sample} - {background_CS} {genotype_CS} vs. {background_S} {genotype_S}".format(**first_row))
        # print(ft_expt)
        
        _plot = plot(
            nbft.fortify(ft_expt, obs=True),
            title=shorten(title_pattern.format(**first_row)), **kwargs) 

        
        fn = file_pattern.format(**first_row).replace(os.path.sep,"_")
        plotpath = plot_path + os.path.sep + fn + '.svg'
        datapath = data_path + os.path.sep + fn + '.csv'
        _plot.save(plotpath)

        df = extract_encoded_data_plotnine(_plot)
        df.to_csv(datapath, index=False)
        
        if show:
            display(_plot)
        plot_paths.append(plotpath)
        data_paths.append(datapath)
        
    return plot_paths, data_paths

def plot_expt_bar(ex, expt, library, predicates='', space='cdr3', layers = [], **kwargs):
    # expt_samples = ex.fts[space].obs.query(f"expt == '{expt}' & phage_library == '{library}'")['sample'].unique()
    
    def do_plot_bar(df, title=None, **kwargs):
        gg = (nvs.top_asv_barplot(df, feature_name=space.upper(), **kwargs) 
              + ggtitle(title) 
              + theme(figure_size=(2,2)))
        gg += layers
        # gg.draw()
        return gg
        
    return _plot_expt(ex, expt, library=library, predicates=predicates, space=space, plot=do_plot_bar, **kwargs)


In [13]:
layers = [
    scale_x_continuous(expand=(0,0)),
    scale_y_continuous(expand=(0,0)),
    theme_bw(), 
    theme(
        axis_title_x=element_blank(),
        axis_title_y=element_blank(),
        figure_size=(2,1.5),
        legend_text=element_text(margin=dict(l=0.5)),
        legend_key_height=3,
        legend_key_width=3,
        legend_entry_spacing_y=0,
        # legend_entry_spacing_x=0.5,
        text=element_text(family='sans', size=5)
    )
]

In [14]:
%%bash
out="results/plots/barplots/arrayed"
rm -rf "$out"
mkdir -p "$out"

out="results/tables/figures/barplots/arrayed"
rm -rf "$out"
mkdir -p "$out"

In [15]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    plot_paths, data_paths = plot_expt_bar(
        ex,'027i','alpaca', 
        layers=layers, 
        # show=True, 
        show=False, 
        plot_path="results/plots/barplots/arrayed",
        data_path="results/tables/figures/barplots/arrayed",
        title_pattern="{sample}: {background_CS} {genotype_CS} vs.\n{background_S} {genotype_S} ({antigen})",
        file_pattern="{sample}_{antigen}",
    )



['1.A1', '1.A2', '1.A3', '1.A4', '1.A5', '1.A6', '1.A7', '1.A8', '1.A9', '1.A10', '1.A11', '1.A12', '1.B1', '1.B2', '1.B3', '1.B4', '1.B5', '1.B6', '1.B7', '1.B8', '1.B9', '1.B10', '1.B11', '1.B12', '1.C1', '1.C2', '1.C3', '1.C4', '1.C5', '1.C6', '1.C7', '1.C8', '1.C9', '1.C10', '1.C11', '1.C12', '1.D1', '1.D2', '1.D3', '1.D4', '1.D5', '1.D6', '1.D7', '1.D8', '1.D9', '1.D10', '1.D11', '1.D12', '1.E1', '1.E2', '1.E3', '1.E4', '1.E5', '1.E6', '1.E7', '1.E8', '1.E9', '1.E10', '1.E11', '1.E12', '1.F1', '1.F2', '1.F3', '1.F4', '1.F5', '1.F6', '1.F7', '1.F8', '1.F9', '1.F10', '1.F11', '1.F12', '1.G1', '1.G2', '1.G3', '1.G4', '1.G5', '1.G6', '1.G7', '1.G8', '1.G9', '1.G10', '1.G11', '1.G12', '1.H1', '1.H2', '1.H3', '1.H4', '1.H5', '1.H6', '1.H7', '1.H8', '1.H9', '1.H10', '1.H11', '1.H12', '2.A1', '2.A2', '2.A3', '2.A4', '2.A5', '2.A6', '2.A7', '2.A8', '2.A9', '2.A10', '2.A11', '2.A12', '2.B1', '2.B2', '2.B3', '2.B4', '2.B5', '2.B6', '2.B7', '2.B8', '2.B9', '2.B10', '2.B11', '2.B12', '2.C1', '

In [16]:
arrayplot(plot_paths, 'results/plots/barplots/arrayed_{page}.svg')

results/plots/barplots/arrayed_1.svg
results/plots/barplots/arrayed_2.svg
results/plots/barplots/arrayed_3.svg
results/plots/barplots/arrayed_4.svg


In [17]:
def compile_data_frames(paths, output):
    dfs = []
    for path in paths:
        dfs.append(pd.read_csv(path))
    pd.concat(dfs).to_csv(output, index=False)

In [18]:
compile_data_frames(data_paths, 'results/tables/figures/barplots/arrayed.csv')

## Diversity of high-throughput cell-based pannings using alpaca library (`fig-suppl-arrayed-rank-abundance`)

In [19]:
def whittaker(ex, expt, library, space='cdr3', predicates='', **kwargs):
    query = f"expt == '{expt}' & phage_library == '{library}' & replicate == 1 & io == 'i' & kind == '+'"
    if predicates != '':
        query += f" & ({predicates})"
    ft = ex.query(query, space=space)

    return nvs.rank_abundance_plot(ft, **kwargs)

In [20]:
def plot_expt_whittaker(
    ex, expt, library, predicates='', space='cdr3', layers = [],
    path='.',
    plot_path="results/plots/barplots/arrayed", 
    data_path="results/tables/figures/barplots/arrayed",
    title_pattern="{background_S} {antigen} - {method}",
    file_pattern="{background_S}_{antigen}_{method}",
    show=True, sort=True,
    last_round=5, **kwargs):
    
    query = f"expt == '{expt}' & phage_library == '{library}' & replicate == 1 & io == 'i' & kind == '+'"
    if predicates != '':
        query += f" & ({predicates})"
    ft = ex.query(query, space=space)
    expt_samples = ft.obs['sample'].unique()
    if sort:
        from natsort import natsorted
        expt_samples = natsorted(expt_samples)
        
    print(expt_samples)
    
    plot_paths = []
    data_paths = []
    for sample in expt_samples:
        first_row = ex.obs.query(f"({query}) & (sample == '{sample}')").iloc[0,:]
            
        _plot = (whittaker(
            ex, expt, library, space=space, predicates = f"(sample == '{sample}')", 
            n_sample=dict(n=100, replace=True),
            line=dict(size=1),
            color="r", group="r") 
                 + ggtitle(shorten(title_pattern.format(**first_row))) 
        + layers)


        import os
        fn = file_pattern.format(**first_row).replace(os.path.sep,"_")
        
        plotpath = plot_path + os.path.sep + fn + '.svg'
        _plot.save(plotpath)
        
        datapath = data_path + os.path.sep + fn + '.csv'
        df = extract_encoded_data_plotnine(_plot)
        df.to_csv(datapath, index=False)
        
        plot_paths.append(plotpath)
        data_paths.append(datapath)
        
        if show:
            display(_plot)
        
    return plot_paths, data_paths

In [21]:
from matplotlib.ticker import EngFormatter
fmt = EngFormatter(sep="")

layers = [
    scale_color_cmap("viridis"),
    scale_x_continuous(name="feature rank", labels=lambda x: list(map(fmt.format_eng, x))),
    # facet_wrap("desc_ag_short", nrow=1),
    theme_bw(), 
    theme(
        figure_size=(1.5,1.5),
        legend_position='none',
        legend_key_height=4,
        legend_key_width=4,
        text=element_text(family='sans', size=5)
    ),
    guides(color=guide_legend(title="round"))
]

In [22]:
%%bash
out="results/plots/rank_abundance/arrayed"
rm -rf "$out"
mkdir -p "$out"

out="results/tables/figures/rank_abundance/arrayed"
rm -rf "$out"
mkdir -p "$out"

In [23]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    
    plot_paths, data_paths = plot_expt_whittaker(ex,'027i','alpaca', layers=layers, 
                      show=False, 
                      plot_path="results/plots/rank_abundance/arrayed",
                      data_path="results/tables/figures/rank_abundance/arrayed",
                      title_pattern="{sample}: {background_CS} {genotype_CS} vs.\n{background_S} {genotype_S} ({antigen})",
                      file_pattern="{sample}_{antigen}",
                     )

['1.A1', '1.A2', '1.A3', '1.A4', '1.A5', '1.A6', '1.A7', '1.A8', '1.A9', '1.A10', '1.A11', '1.A12', '1.B1', '1.B2', '1.B3', '1.B4', '1.B5', '1.B6', '1.B7', '1.B8', '1.B9', '1.B10', '1.B11', '1.B12', '1.C1', '1.C2', '1.C3', '1.C4', '1.C5', '1.C6', '1.C7', '1.C8', '1.C9', '1.C10', '1.C11', '1.C12', '1.D1', '1.D2', '1.D3', '1.D4', '1.D5', '1.D6', '1.D7', '1.D8', '1.D9', '1.D10', '1.D11', '1.D12', '1.E1', '1.E2', '1.E3', '1.E4', '1.E5', '1.E6', '1.E7', '1.E8', '1.E9', '1.E10', '1.E11', '1.E12', '1.F1', '1.F2', '1.F3', '1.F4', '1.F5', '1.F6', '1.F7', '1.F8', '1.F9', '1.F10', '1.F11', '1.F12', '1.G1', '1.G2', '1.G3', '1.G4', '1.G5', '1.G6', '1.G7', '1.G8', '1.G9', '1.G10', '1.G11', '1.G12', '1.H1', '1.H2', '1.H3', '1.H4', '1.H5', '1.H6', '1.H7', '1.H8', '1.H9', '1.H10', '1.H11', '1.H12', '2.A1', '2.A2', '2.A3', '2.A4', '2.A5', '2.A6', '2.A7', '2.A8', '2.A9', '2.A10', '2.A11', '2.A12', '2.B1', '2.B2', '2.B3', '2.B4', '2.B5', '2.B6', '2.B7', '2.B8', '2.B9', '2.B10', '2.B11', '2.B12', '2.C1', '

In [24]:
arrayplot(plot_paths, 'results/plots/rank_abundance/arrayed_{page}.svg')

results/plots/rank_abundance/arrayed_1.svg
results/plots/rank_abundance/arrayed_2.svg
results/plots/rank_abundance/arrayed_3.svg
results/plots/rank_abundance/arrayed_4.svg


In [25]:
compile_data_frames(data_paths, 'results/tables/figures/rank_abundance/arrayed.csv')